In [1]:
import glob
import os

eval_folder = "/homes/kb658/fusion/evaluations/"

all_json_files = glob.glob(os.path.join(eval_folder, "**/*.json"), recursive=True)

## Comparison with AMT

In [16]:
experiment_5_files = [f for f in all_json_files if "experiment_5" in f]
experiment_5_directories = set([os.path.dirname(f) for f in experiment_5_files])

experiment_6_files = [f for f in all_json_files if "experiment_6" in f]
experiment_6_directories = set([os.path.dirname(f) for f in experiment_6_files])

In [3]:
import json
import math
import pprint

def get_metrics_from_file(filepaths):
    results = {'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'pitch_class_kl': [], 
            'pctm_cosine_sim': [], 
            'avg_ioi': []}

    for f in filepaths:
        with open(f, "r") as f:
            metrics = json.load(f)

        for k in results.keys():
            results[k].append(metrics[k])

    # Average num notes, note density, used pitch, ioi element wise in list
    avg_num_notes_original = sum([i[0] for i in results['num_notes']]) / len(results['num_notes'])
    avg_num_notes_generated = sum([i[1] for i in results['num_notes']]) / len(results['num_notes'])

    avg_note_density_original = sum([i[0] for i in results['note_density']]) / len(results['note_density'])
    avg_note_density_generated = sum([i[1] for i in results['note_density']]) / len(results['note_density'])

    avg_used_pitch_original = sum([i[0] for i in results['used_pitch']]) / len(results['used_pitch'])
    avg_used_pitch_generated = sum([i[1] for i in results['used_pitch']]) / len(results['used_pitch'])

    avg_ioi_original = sum([i[0] for i in results['avg_ioi']]) / len(results['avg_ioi'])
    avg_ioi_generated = sum([i[1] for i in results['avg_ioi']]) / len(results['avg_ioi'])

    avg_pitch_class_kl = sum([i for i in results['pitch_class_kl']]) / len(results['pitch_class_kl'])

    # Filter out NaN values
    results['pctm_cosine_sim'] = [i for i in results['pctm_cosine_sim'] if not math.isnan(i)]
    avg_pctm_cosine_sim = sum([i for i in results['pctm_cosine_sim']]) / len(results['pctm_cosine_sim'])

    # Add averages to results
    results = {
        'num_notes': (avg_num_notes_original, avg_num_notes_generated),
        'note_density': (avg_note_density_original, avg_note_density_generated),
        'used_pitch': (avg_used_pitch_original, avg_used_pitch_generated),
        'pitch_class_kl': avg_pitch_class_kl,
        'pctm_cosine_sim': avg_pctm_cosine_sim,
        'avg_ioi': (avg_ioi_original, avg_ioi_generated)
    }

    return results

## Infilling

In [4]:
amt_filepaths = [os.path.join(f, "amt_metrics.json") for f in experiment_5_directories]
improvnet_filepaths = [os.path.join(f, "metrics.json") for f in experiment_5_directories]

amt_metrics = get_metrics_from_file(amt_filepaths)
improvnet_metrics = get_metrics_from_file(improvnet_filepaths)

pprint.pprint(amt_metrics)
pprint.pprint(improvnet_metrics)

{'avg_ioi': (0.13942976033627477, 0.1507879825995296),
 'note_density': (32.437751004016064, 49.626506024096386),
 'num_notes': (87.81927710843374, 99.25301204819277),
 'pctm_cosine_sim': 0.3623992681859348,
 'pitch_class_kl': 1.4593494362096302,
 'used_pitch': (27.879518072289155, 25.44578313253012)}
{'avg_ioi': (0.13942976033627477, 0.12241346016156072),
 'note_density': (32.437751004016064, 36.30120481927711),
 'num_notes': (87.81927710843374, 95.62650602409639),
 'pctm_cosine_sim': 0.40362731442096167,
 'pitch_class_kl': 0.8906669396230866,
 'used_pitch': (27.879518072289155, 28.783132530120483)}


## Prompt Continuation

In [17]:
amt_filepaths = [os.path.join(f, "amt_metrics.json") for f in experiment_6_directories]
improvnet_filepaths = [os.path.join(f, "metrics.json") for f in experiment_6_directories]

amt_metrics = get_metrics_from_file(amt_filepaths)
improvnet_metrics = get_metrics_from_file(improvnet_filepaths)

pprint.pprint(amt_metrics)
pprint.pprint(improvnet_metrics)

{'avg_ioi': (0.14049655073108946, 0.1386016823059197),
 'note_density': (30.84738955823294, 67.66265060240964),
 'num_notes': (84.75903614457832, 129.46987951807228),
 'pctm_cosine_sim': 0.3073806920023673,
 'pitch_class_kl': 1.6083742717460898,
 'used_pitch': (27.072289156626507, 25.156626506024097)}
{'avg_ioi': (0.14049655073108946, 0.12434726927005689),
 'note_density': (30.84738955823294, 37.484939759036145),
 'num_notes': (84.75903614457832, 96.6867469879518),
 'pctm_cosine_sim': 0.34696265850934427,
 'pitch_class_kl': 1.250027750177615,
 'used_pitch': (27.072289156626507, 28.53012048192771)}


# Style Transfer

In [2]:
import json
import math
import pprint

def get_metrics_from_file(filepaths):
    results = {'Original Genre Probabilities': [], 
            'Generated Genre Probabilities': [], 
            'SSM Similarity Score': [], 
            'Chroma Frame Similarity Score': []}

    for f in filepaths:
        with open(f, "r") as f:
            metrics = json.load(f)

        for k in results.keys():
            if metrics[k] is not None:
                results[k].append(metrics[k])
            # else:
            #     print(f)

    # Average classical and jazz probabilities, ssm similarity score, chroma frame similarity score element wise in list
    avg_classical_original = sum([i['Classical'] for i in results['Original Genre Probabilities']]) / len(results['Original Genre Probabilities'])
    avg_jazz_original = sum([i['Jazz'] for i in results['Original Genre Probabilities']]) / len(results['Original Genre Probabilities'])
    avg_classical_generated = sum([i['Classical'] for i in results['Generated Genre Probabilities']]) / len(results['Generated Genre Probabilities'])
    avg_jazz_generated = sum([i['Jazz'] for i in results['Generated Genre Probabilities']]) / len(results['Generated Genre Probabilities'])

    avg_ssm_similarity_score_mse = sum([i['SSM MSE'] for i in results['SSM Similarity Score']]) / len(results['SSM Similarity Score'])
    avg_ssm_similarity_score_pearson = sum([i['SSM Pearson Correlation'] for i in results['SSM Similarity Score']]) / len(results['SSM Similarity Score'])
    avg_ssm_similarity_cosine = sum([i['SSM Cosine Similarity'] for i in results['SSM Similarity Score']]) / len(results['SSM Similarity Score'])

    avg_chroma_frame_similarity_score = sum([i for i in results['Chroma Frame Similarity Score']]) / len(results['Chroma Frame Similarity Score'])

    # Add averages to results
    results = {
        'Original Genre Probabilities': {'classical': avg_classical_original, 'jazz': avg_jazz_original}, 
        'Generated Genre Probabilities': {'classical': avg_classical_generated, 'jazz': avg_jazz_generated},
        'SSM Similarity Score': {'SSM MSE': avg_ssm_similarity_score_mse, 'SSM Pearson Correlation': avg_ssm_similarity_score_pearson, 'SSM Cosine Similarity': avg_ssm_similarity_cosine},
        'Chroma Frame Similarity Score': avg_chroma_frame_similarity_score
    }

    return results

## Corruption Rates

In [32]:
corruption_rates = ["corruption_rate_0.25", "corruption_rate_0.5", "corruption_rate_0.75", "corruption_rate_1.0"]
passes = ["pass_1/", "pass_2/", "pass_3/", "pass_4/", "pass_5/", "pass_6/", "pass_7/", "pass_8/", "pass_9/", "pass_10/"]

for i in ["corruption_rate_0.25", "corruption_rate_0.5", "corruption_rate_0.75", "corruption_rate_1.0"]:
    for k in passes:
        print(i, k)
        experiment_1_files = [f for f in all_json_files if "experiment_1" in f and "classical" in f and "target_style_jazz" in f and i in f and k in f and "harmony" not in f]
        print(len(experiment_1_files))
        style_conversion_metrics = get_metrics_from_file(experiment_1_files)
        pprint.pprint(style_conversion_metrics)

corruption_rate_0.25 pass_1/
270
{'Chroma Frame Similarity Score': 0.6355199809074402,
 'Generated Genre Probabilities': {'classical': 0.7508712486197399,
                                   'jazz': 0.24194202757133343},
 'Original Genre Probabilities': {'classical': 0.7876706539873451,
                                  'jazz': 0.20465879339906995},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9877907514572144,
                          'SSM MSE': 0.009455740102101118,
                          'SSM Pearson Correlation': 0.8547302753820487}}
corruption_rate_0.25 pass_2/
270
{'Chroma Frame Similarity Score': 0.6355953121185303,
 'Generated Genre Probabilities': {'classical': 0.7154746122912294,
                                   'jazz': 0.27771595775067315},
 'Original Genre Probabilities': {'classical': 0.7876296102128156,
                                  'jazz': 0.2046997650083637},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9786423754692077,
                        

In [31]:
corruption_rates = ["corruption_rate_0.25", "corruption_rate_0.5", "corruption_rate_0.75", "corruption_rate_1.0"]
passes = ["pass_1/", "pass_2/", "pass_3/", "pass_4/", "pass_5/", "pass_6/", "pass_7/", "pass_8/", "pass_9/", "pass_10/"]

for i in ["corruption_rate_0.25", "corruption_rate_0.5", "corruption_rate_0.75", "corruption_rate_1.0"]:
    for k in passes:
        print(i, k)
        experiment_1_files = [f for f in all_json_files if "experiment_1" in f and "jazz" in f and "target_style_classical" in f and i in f and k in f and "harmony" not in f]
        print(len(experiment_1_files))
        style_conversion_metrics = get_metrics_from_file(experiment_1_files)
        pprint.pprint(style_conversion_metrics)

corruption_rate_0.25 pass_1/
560
{'Chroma Frame Similarity Score': 0.6332879266945837,
 'Generated Genre Probabilities': {'classical': 0.11211560656299209,
                                   'jazz': 0.8819446950210753},
 'Original Genre Probabilities': {'classical': 0.08836461873258294,
                                  'jazz': 0.906222176596412},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9849507359225323,
                          'SSM MSE': 0.011199709541664526,
                          'SSM Pearson Correlation': 0.8288027867792559}}
corruption_rate_0.25 pass_2/
560
{'Chroma Frame Similarity Score': 0.6330049859248708,
 'Generated Genre Probabilities': {'classical': 0.1327464303920343,
                                   'jazz': 0.8612096135065966},
 'Original Genre Probabilities': {'classical': 0.08852990433277559,
                                  'jazz': 0.9060503243548967},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9744344156430815,
                         

## Context Window

In [30]:
context = ["context_1_1", "context_2_2", "context_3_3", "context_4_4", "context_5_5"]
passes = ["pass_1/", "pass_2/", "pass_3/", "pass_4/", "pass_5/", "pass_6/", "pass_7/", "pass_8/", "pass_9/", "pass_10/"]

for i in context:
    for k in passes:
        print(i, k)
        experiment_1_files = [f for f in all_json_files if "experiment_2" in f and "classical" in f and "target_style_jazz" in f and i in f and k in f and "harmony" not in f]
        print(len(experiment_1_files))
        style_conversion_metrics = get_metrics_from_file(experiment_1_files)
        pprint.pprint(style_conversion_metrics)

context_1_1 pass_1/
27
{'Chroma Frame Similarity Score': 0.6352163910865783,
 'Generated Genre Probabilities': {'classical': 0.5741642893201971,
                                   'jazz': 0.41995514545144264},
 'Original Genre Probabilities': {'classical': 0.7877253305354798,
                                  'jazz': 0.20459247332402672},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9562938523292541,
                          'SSM MSE': 0.03430368825793266,
                          'SSM Pearson Correlation': 0.46693804150857526}}
context_1_1 pass_2/
27
{'Chroma Frame Similarity Score': 0.6363572359085083,
 'Generated Genre Probabilities': {'classical': 0.46984756992614424,
                                   'jazz': 0.5254634276784896},
 'Original Genre Probabilities': {'classical': 0.787723191578933,
                                  'jazz': 0.2045667092172576},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9425159358978271,
                          'SSM MSE': 0.045006

In [33]:
context = ["context_1_1", "context_2_2", "context_3_3", "context_4_4", "context_5_5"]
passes = ["pass_1/", "pass_2/", "pass_3/", "pass_4/", "pass_5/", "pass_6/", "pass_7/", "pass_8/", "pass_9/", "pass_10/"]

for i in context:
    for k in passes:
        print(i, k)
        experiment_1_files = [f for f in all_json_files if "experiment_2" in f and "jazz" in f and "target_style_classical" in f and i in f and k in f and "harmony" not in f]
        print(len(experiment_1_files))
        style_conversion_metrics = get_metrics_from_file(experiment_1_files)
        pprint.pprint(style_conversion_metrics)

context_1_1 pass_1/
56
{'Chroma Frame Similarity Score': 0.629231030290777,
 'Generated Genre Probabilities': {'classical': 0.2368307823158963,
                                   'jazz': 0.7562456157621672},
 'Original Genre Probabilities': {'classical': 0.08818854913090446,
                                  'jazz': 0.9063082991647841},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9435269615866921,
                          'SSM MSE': 0.04204798755659298,
                          'SSM Pearson Correlation': 0.3628145074962529}}
context_1_1 pass_2/
56
{'Chroma Frame Similarity Score': 0.6270948594266718,
 'Generated Genre Probabilities': {'classical': 0.33077959160747816,
                                   'jazz': 0.6634179121517381},
 'Original Genre Probabilities': {'classical': 0.08786524778475817,
                                  'jazz': 0.906859254327589},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9315921154889193,
                          'SSM MSE': 0.05123130

## Number of passes and type of corruptions

In [4]:
corruption_rates = ["corruption_rate_1.0"]
corruption_type = ["fragmentation", "incorrect_transposition", "note_modification", "onset_duration_mask", "permute_pitch_velocity", "permute_pitches", "pitch_velocity_mask", "skyline", "whole_mask"]
passes = ["pass_1/", "pass_2/", "pass_3/", "pass_4/", "pass_5/", "pass_6/", "pass_7/", "pass_8/", "pass_9/", "pass_10/"]

for c in corruption_type:
    for i in corruption_rates:
        for k in passes:
            print(c, i, k)
            experiment_1_files = [f for f in all_json_files if "experiment_1" in f and "classical" in f and "target_style_jazz" in f and i in f and k in f and c in f and "harmony" not in f]
            print(len(experiment_1_files))
            style_conversion_metrics = get_metrics_from_file(experiment_1_files)
            pprint.pprint(style_conversion_metrics)

fragmentation corruption_rate_1.0 pass_1/
27
{'Chroma Frame Similarity Score': 0.6368438959121704,
 'Generated Genre Probabilities': {'classical': 0.5021366598815801,
                                   'jazz': 0.49327559110430647},
 'Original Genre Probabilities': {'classical': 0.7876144619031884,
                                  'jazz': 0.20467718440511698},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9521013808250427,
                          'SSM MSE': 0.0374980353564024,
                          'SSM Pearson Correlation': 0.41595248496798254}}
fragmentation corruption_rate_1.0 pass_2/
26
{'Chroma Frame Similarity Score': 0.6368129327893257,
 'Generated Genre Probabilities': {'classical': 0.4532735200689478,
                                   'jazz': 0.5419571729511549},
 'Original Genre Probabilities': {'classical': 0.7905933670452575,
                                  'jazz': 0.20187185403412403},
 'SSM Similarity Score': {'SSM Cosine Similarity': 0.9468424891432127,
 

## Harmony

In [2]:
import json
import pprint

def get_harmony_metrics(harmony_files):

    generated_results = {
            'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'pitch_class_kl': [], 
            'pctm_cosine_sim': [], 
            'avg_ioi': [],
            'n_pitches': [],
            'n_unique_chords': [],
            'pitch_in_scale_rate': [],
            'pitch_range': [],
            'polyphony': [],
            'polyphony_rate': [],
            'groove_consistency': [],
            'tonal_tension_diameter': [],
            'tonal_tension_tensile': [],
            'tonal_tension_centroid': []
            }

    original_results = {
            'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'avg_ioi': [],
            'n_pitches': [],
            'n_unique_chords': [],
            'pitch_in_scale_rate': [],
            'pitch_range': [],
            'polyphony': [],
            'polyphony_rate': [],
            'groove_consistency': [],
            'tonal_tension_diameter': [],
            'tonal_tension_tensile': [],
            'tonal_tension_centroid': []
            }
    
    monophonic_results = {
            'num_notes': [], 
            'note_density': []
            }
    
    for file in harmony_files:
        with open(file, "r") as f:
            metrics = json.load(f)

        for k in generated_results.keys():
            if k in metrics['Metric Pairs']:
                if type(metrics['Metric Pairs'][k]) == list:
                    generated_results[k].append(metrics['Metric Pairs'][k][1])
                else:
                    generated_results[k].append(metrics['Metric Pairs'][k])
            elif k in metrics['Generated MusPy Metrics']["pitch_related"]:
                generated_results[k].append(metrics['Generated MusPy Metrics']["pitch_related"][k])
            elif k in metrics['Generated MusPy Metrics']["rhythm_related"]:
                generated_results[k].append(metrics['Generated MusPy Metrics']["rhythm_related"][k])
            elif "centroid" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Diameter"])
            elif "diameter" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Tensile"])
            elif "tensile" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Centroid Diff"])

        for k in original_results.keys():
            if k in metrics['Metric Pairs']:
                if type(metrics['Metric Pairs'][k]) == list:
                    original_results[k].append(metrics['Metric Pairs'][k][0])
            elif k in metrics['Original MusPy Metrics']["pitch_related"]:
                original_results[k].append(metrics['Original MusPy Metrics']["pitch_related"][k])
            elif k in metrics['Original MusPy Metrics']["rhythm_related"]:
                original_results[k].append(metrics['Original MusPy Metrics']["rhythm_related"][k])
            elif "centroid" in k:
                original_results[k].append(metrics["Original Tonal Tension Diameter"])
            elif "diameter" in k:
                original_results[k].append(metrics["Original Tonal Tension Tensile"])
            elif "tensile" in k:
                original_results[k].append(metrics["Original Tonal Tension Centroid Diff"])

        for k in monophonic_results.keys():
            monophonic_results[k].append(metrics['Monophonic Metric Pairs'][k][0])

    # Average all values in list
    for k in generated_results.keys():
        generated_results[k] = sum(generated_results[k]) / len(generated_results[k])
    for k in original_results.keys():
        original_results[k] = sum(original_results[k]) / len(original_results[k])
    for k in monophonic_results.keys():
        monophonic_results[k] = sum(monophonic_results[k]) / len(monophonic_results[k])
                
    return generated_results, original_results, monophonic_results

### Classical

In [3]:
harmony_files = [f for f in all_json_files if "harmony" in f and "classical" in f]
random_chords_files_constraints = [f for f in harmony_files if "random_chords" in f and "constraints" in f]
generated_files_constraints = [f for f in harmony_files if "random_chords" not in f and "with_constraints" in f]
generated_files_no_constraints = [f for f in harmony_files if "random_chords" not in f and "without_constraints" in f]

# Print length of all files
print(len(harmony_files))
print(len(random_chords_files_constraints))
print(len(generated_files_constraints))
print(len(generated_files_no_constraints))

1745
580
580
585


In [4]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.15442037038237283,
 'groove_consistency': 0.9919458949810027,
 'n_pitches': 35.82586206896552,
 'n_unique_chords': 18.30862068965517,
 'note_density': 35.952364957892826,
 'num_notes': 586.0155172413793,
 'pctm_cosine_sim': 0.7831491786421979,
 'pitch_class_kl': 0.3307160384179918,
 'pitch_in_scale_rate': 0.8016703430999436,
 'pitch_range': 49.29137931034483,
 'polyphony': 10.672827605167724,
 'polyphony_rate': 0.911866069708051,
 'tonal_tension_centroid': 1.8638217622781448,
 'tonal_tension_diameter': 0.623134855458608,
 'tonal_tension_tensile': 0.5469808436029697,
 'used_pitch': 35.82586206896552}
{'avg_ioi': 0.28653001901642006,
 'groove_consistency': 0.9998109796008848,
 'n_pitches': 25.862068965517242,
 'n_unique_chords': 13.293103448275861,
 'note_density': 19.201245679514017,
 'num_notes': 304.17241379310343,
 'pitch_in_scale_rate': 0.7940400958441709,
 'pitch_range': 34.689655172413794,
 'polyphony': 13.328439371162316,
 'polyphony_rate': 0.9576767930464862,
 'ton

In [5]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_no_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.47616281960577156,
 'groove_consistency': 0.9960447409479711,
 'n_pitches': 18.242735042735042,
 'n_unique_chords': 13.62051282051282,
 'note_density': 11.067116856486097,
 'num_notes': 166.45811965811967,
 'pctm_cosine_sim': 0.7251719505215872,
 'pitch_class_kl': 0.5649605433236629,
 'pitch_in_scale_rate': 0.8026301232365253,
 'pitch_range': 28.96923076923077,
 'polyphony': 2.4244561935319395,
 'polyphony_rate': 0.24573336007215227,
 'tonal_tension_centroid': 0.41776751121499106,
 'tonal_tension_diameter': 0.7473532016353547,
 'tonal_tension_tensile': 0.7702037484038985,
 'used_pitch': 18.242735042735042}
{'avg_ioi': 0.2888099221746096,
 'groove_consistency': 0.9998117128673026,
 'n_pitches': 25.82905982905983,
 'n_unique_chords': 13.264957264957266,
 'note_density': 19.107217938663474,
 'num_notes': 302.62393162393164,
 'pitch_in_scale_rate': 0.7950462890923603,
 'pitch_range': 34.68376068376068,
 'polyphony': 13.306769833359867,
 'polyphony_rate': 0.9578652796848044,
 

In [6]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(random_chords_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.15442037038237283,
 'groove_consistency': 0.9919455312871803,
 'n_pitches': 55.28793103448276,
 'n_unique_chords': 37.38620689655173,
 'note_density': 35.952364957892826,
 'num_notes': 586.0155172413793,
 'pctm_cosine_sim': 0.6264484805888032,
 'pitch_class_kl': 0.30400444221038014,
 'pitch_in_scale_rate': 0.6364044324823606,
 'pitch_range': 62.19310344827586,
 'polyphony': 5.100670359272729,
 'polyphony_rate': 0.7302690584025148,
 'tonal_tension_centroid': 2.782422818446298,
 'tonal_tension_diameter': 0.9120607382553692,
 'tonal_tension_tensile': 0.5321973318983758,
 'used_pitch': 55.28793103448276}
{'avg_ioi': 0.28653001901642006,
 'groove_consistency': 0.9998109796008848,
 'n_pitches': 25.862068965517242,
 'n_unique_chords': 13.293103448275861,
 'note_density': 19.201245679514017,
 'num_notes': 304.17241379310343,
 'pitch_in_scale_rate': 0.7940400958441709,
 'pitch_range': 34.689655172413794,
 'polyphony': 13.328439371162316,
 'polyphony_rate': 0.9576767930464862,
 'to

### Jazz

In [7]:
harmony_files = [f for f in all_json_files if "harmony" in f and "jazz" in f]
random_chords_files_constraints = [f for f in harmony_files if "random_chords" in f and "constraints" in f]
generated_files_constraints = [f for f in harmony_files if "random_chords" not in f and "with_constraints" in f]
generated_files_no_constraints = [f for f in harmony_files if "random_chords" not in f and "without_constraints" in f]

# Print length of all files
print(len(harmony_files))
print(len(random_chords_files_constraints))
print(len(generated_files_constraints))
print(len(generated_files_no_constraints))

1755
585
585
585


In [8]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.16470294157811796,
 'groove_consistency': 0.9917543830518143,
 'n_pitches': 37.957264957264954,
 'n_unique_chords': 21.246153846153845,
 'note_density': 34.487780488162876,
 'num_notes': 556.3230769230769,
 'pctm_cosine_sim': 0.7716522655264306,
 'pitch_class_kl': 0.3551746569700624,
 'pitch_in_scale_rate': 0.780263087506738,
 'pitch_range': 53.37094017094017,
 'polyphony': 10.807139433330866,
 'polyphony_rate': 0.9225642639211044,
 'tonal_tension_centroid': 2.1117157961713304,
 'tonal_tension_diameter': 0.6271673410988967,
 'tonal_tension_tensile': 0.5207329033924408,
 'used_pitch': 37.957264957264954}
{'avg_ioi': 0.2888099221746096,
 'groove_consistency': 0.9998117128673026,
 'n_pitches': 25.82905982905983,
 'n_unique_chords': 13.264957264957266,
 'note_density': 19.107217938663474,
 'num_notes': 302.62393162393164,
 'pitch_in_scale_rate': 0.7950462890923603,
 'pitch_range': 34.68376068376068,
 'polyphony': 13.306769833359867,
 'polyphony_rate': 0.9578652796848044,
 'to

In [9]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_no_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.47847792698090486,
 'groove_consistency': 0.9958869436783827,
 'n_pitches': 18.415384615384614,
 'n_unique_chords': 14.403418803418804,
 'note_density': 11.308143826810042,
 'num_notes': 173.9965811965812,
 'pctm_cosine_sim': 0.7207078263228398,
 'pitch_class_kl': 0.5261224313030578,
 'pitch_in_scale_rate': 0.8007234700880137,
 'pitch_range': 29.851282051282052,
 'polyphony': 2.4561393029298904,
 'polyphony_rate': 0.23366606715765992,
 'tonal_tension_centroid': 0.49445170076147965,
 'tonal_tension_diameter': 0.7639787577064305,
 'tonal_tension_tensile': 0.78056169849884,
 'used_pitch': 18.415384615384614}
{'avg_ioi': 0.2888099221746096,
 'groove_consistency': 0.9998117128673026,
 'n_pitches': 25.82905982905983,
 'n_unique_chords': 13.264957264957266,
 'note_density': 19.107217938663474,
 'num_notes': 302.62393162393164,
 'pitch_in_scale_rate': 0.7950462890923603,
 'pitch_range': 34.68376068376068,
 'polyphony': 13.306769833359867,
 'polyphony_rate': 0.9578652796848044,
 '

In [10]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(random_chords_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.16470294157811796,
 'groove_consistency': 0.9917538547748713,
 'n_pitches': 55.712820512820514,
 'n_unique_chords': 37.67863247863248,
 'note_density': 34.494357614740004,
 'num_notes': 556.3230769230769,
 'pctm_cosine_sim': 0.6312525033268912,
 'pitch_class_kl': 0.33595147528204783,
 'pitch_in_scale_rate': 0.6395219272574528,
 'pitch_range': 65.2991452991453,
 'polyphony': 5.431145510938808,
 'polyphony_rate': 0.7767833163640842,
 'tonal_tension_centroid': 2.9386401972962912,
 'tonal_tension_diameter': 0.9157302411204953,
 'tonal_tension_tensile': 0.5123366135353165,
 'used_pitch': 55.712820512820514}
{'avg_ioi': 0.2888099221746096,
 'groove_consistency': 0.9998117128673026,
 'n_pitches': 25.82905982905983,
 'n_unique_chords': 13.264957264957266,
 'note_density': 19.107217938663474,
 'num_notes': 302.62393162393164,
 'pitch_in_scale_rate': 0.7950462890923603,
 'pitch_range': 34.68376068376068,
 'polyphony': 13.306769833359867,
 'polyphony_rate': 0.9578652796848044,
 'ton